In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader


In [2]:
# import os

# # Path to the file you want to delete
# file_path = '/kaggle/working/overfit.pth.tar'

# # Delete the file
# os.remove(file_path)


# UTILS

In [3]:
def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)


def load_checkpoint(checkpoint, model, optimizer):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

In [4]:
def intersection_over_union(boxes_preds, boxes_labels, box_format="corners"):
    # Boxes pred shape(N,4) N - number of bounding boxes
    #boxes label shaoe is  (N,4)

    if box_format=="midpoint":
        box1_x1=boxes_preds[...,0:1] - boxes_preds[...,2:3]/2
        box1_y1=boxes_preds[...,1:2] - boxes_preds[...,3:4] /2
        box1_x2=boxes_preds[...,2:3] +boxes_preds[...,2:3]/2
        box1_y2=boxes_preds[...,3:4] +boxes_preds[...,3:4] /2        #(N,1)
        box2_x1=boxes_labels[...,0:1] - boxes_labels[...,2:3]/2
        box2_y1=boxes_labels[...,1:2] - boxes_labels[...,3:4] /2
        box2_x2=boxes_labels[...,2:3] +boxes_labels[...,2:3]/2
        box2_y2=boxes_labels[...,3:4] +boxes_labels[...,3:4] /2

        
    elif box_format=="corners":
        box1_x1=boxes_preds[...,0:1]
        box1_y1=boxes_preds[...,1:2]
        box1_x2=boxes_preds[...,2:3]
        box1_y2=boxes_preds[...,3:4] #(N,1)
        box2_x1=boxes_labels[...,0:1]
        box2_y1=boxes_labels[...,1:2]
        box2_x2=boxes_labels[...,2:3]
        box2_y2=boxes_labels[...,3:4] 

    x1= torch.max(box1_x1,box2_x1)
    y1= torch.max(box1_y1,box2_y1)
    x2= torch.min(box1_x2,box2_x2)
    y2= torch.min(box1_y2,box2_y2)

    #.clamp(0)- when they don't intersect
    intersection= (x2-x1).clamp(0)*(y2-y1).clamp(0)

    box1_area= abs(box1_x2-box1_x1)*abs(box1_y2-box1_y1)
    box2_area= abs(box2_x2-box2_x1)*abs(box2_y2-box2_y1)

    union = box1_area + box2_area - intersection + 1e-8

    iou= intersection/union
    return iou
    


In [5]:
import torch

def non_max_suppression(bboxes,iou_threshold,threshold,box_format='corners'):
    #prediction=[[1,0.6,x1,y1,x2,y2],[],[]] 1- class-->argmax of all the probab (class logits) from the model
    assert type(bboxes)==list
    bboxes= [box for box in bboxes if box[1]>threshold]
    bboxes=sorted(bboxes,key=lambda x: x[1],reverse=True) #sorting with the highest prob
    bboxes_after_nms=[]
    
    while bboxes:
        chosen_box=bboxes.pop(0)
        bboxes=[
            box for box in bboxes if 
            box[0]!=chosen_box[0] or 
            intersection_over_union(torch.tensor(chosen_box[2:]),
            torch.tensor(box[2:]),
            box_format=box_format,) <iou_threshold #keep the box if it's not overlapping too much
            
        ]
        bboxes_after_nms.append(chosen_box)
    return bboxes_after_nms
    
        

In [6]:
from collections import Counter


In [7]:
def mean_average_precision(
    pred_boxes, true_boxes, iou_threshold=0.5, box_format="midpoint", num_classes=20
):
   
    
    average_precisions = []

    
    epsilon = 1e-6

    for c in range(num_classes):
        detections = []
        ground_truths = []

        
        for detection in pred_boxes:
            if detection[1] == c:
                detections.append(detection)

        for true_box in true_boxes:
            if true_box[1] == c:
                ground_truths.append(true_box)

        
        amount_bboxes = Counter([gt[0] for gt in ground_truths])

        
        for key, val in amount_bboxes.items():
            amount_bboxes[key] = torch.zeros(val)

        
        detections.sort(key=lambda x: x[2], reverse=True)
        TP = torch.zeros((len(detections)))
        FP = torch.zeros((len(detections)))
        total_true_bboxes = len(ground_truths)
        
        
        if total_true_bboxes == 0:
            continue

        for detection_idx, detection in enumerate(detections):
            
            ground_truth_img = [
                bbox for bbox in ground_truths if bbox[0] == detection[0]
            ]

            num_gts = len(ground_truth_img)
            best_iou = 0

            for idx, gt in enumerate(ground_truth_img):
                iou = intersection_over_union(
                    torch.tensor(detection[3:]),
                    torch.tensor(gt[3:]),
                    box_format=box_format,
                )

                if iou > best_iou:
                    best_iou = iou
                    best_gt_idx = idx

            if best_iou > iou_threshold:
                
                if amount_bboxes[detection[0]][best_gt_idx] == 0:
                    
                    TP[detection_idx] = 1
                    amount_bboxes[detection[0]][best_gt_idx] = 1
                else:
                    FP[detection_idx] = 1

            
            else:
                FP[detection_idx] = 1

        TP_cumsum = torch.cumsum(TP, dim=0)
        FP_cumsum = torch.cumsum(FP, dim=0)
        recalls = TP_cumsum / (total_true_bboxes + epsilon)
        precisions = torch.divide(TP_cumsum, (TP_cumsum + FP_cumsum + epsilon))
        precisions = torch.cat((torch.tensor([1]), precisions))
        recalls = torch.cat((torch.tensor([0]), recalls))
        # torch.trapz for numerical integration
        average_precisions.append(torch.trapz(precisions, recalls))

    return sum(average_precisions) / len(average_precisions)

In [8]:
def plot_image(image, boxes):
    """Plots predicted bounding boxes on the image"""
    im = np.array(image)
    height, width, _ = im.shape

    # Create figure and axes
    fig, ax = plt.subplots(1)
    # Display the image
    ax.imshow(im)

    # box[0] is x midpoint, box[2] is width
    # box[1] is y midpoint, box[3] is height

    # Create a Rectangle potch
    for box in boxes:
        box = box[2:]
        assert len(box) == 4, "Got more values than in x, y, w, h, in a box!"
        upper_left_x = box[0] - box[2] / 2
        upper_left_y = box[1] - box[3] / 2
        rect = patches.Rectangle(
            (upper_left_x * width, upper_left_y * height),
            box[2] * width,
            box[3] * height,
            linewidth=1,
            edgecolor="r",
            facecolor="none",
        )
        # Add the patch to the Axes
        ax.add_patch(rect)

    plt.show()

In [9]:
def get_bboxes(
    loader,
    model,
    iou_threshold,
    threshold,
    pred_format="cells",
    box_format="midpoint",
    device="cuda",
):
    all_pred_boxes = []
    all_true_boxes = []

    # make sure model is in eval before get bboxes
    model.eval()
    train_idx = 0

    for batch_idx, (x, labels) in enumerate(loader):
        x = x.to(device)
        labels = labels.to(device)

        with torch.no_grad():
            predictions = model(x)

        batch_size = x.shape[0]
        true_bboxes = cellboxes_to_boxes(labels)
        bboxes = cellboxes_to_boxes(predictions)

        for idx in range(batch_size):
            nms_boxes = non_max_suppression(
                bboxes[idx],
                iou_threshold=iou_threshold,
                threshold=threshold,
                box_format=box_format,
            )


            #if batch_idx == 0 and idx == 0:
            #    plot_image(x[idx].permute(1,2,0).to("cpu"), nms_boxes)
            #    print(nms_boxes)

            for nms_box in nms_boxes:
                all_pred_boxes.append([train_idx] + nms_box)

            for box in true_bboxes[idx]:
                # many will get converted to 0 pred
                if box[1] > threshold:
                    all_true_boxes.append([train_idx] + box)

            train_idx += 1

    model.train()
    return all_pred_boxes, all_true_boxes


In [10]:
def convert_cellboxes(predictions, S=7):
    

    predictions = predictions.to("cpu")
    batch_size = predictions.shape[0]
    predictions = predictions.reshape(batch_size, 7, 7, 30)
    bboxes1 = predictions[..., 21:25]
    bboxes2 = predictions[..., 26:30]
    scores = torch.cat(
        (predictions[..., 20].unsqueeze(0), predictions[..., 25].unsqueeze(0)), dim=0
    )
    best_box = scores.argmax(0).unsqueeze(-1)
    best_boxes = bboxes1 * (1 - best_box) + best_box * bboxes2
    cell_indices = torch.arange(7).repeat(batch_size, 7, 1).unsqueeze(-1)
    x = 1 / S * (best_boxes[..., :1] + cell_indices)
    y = 1 / S * (best_boxes[..., 1:2] + cell_indices.permute(0, 2, 1, 3))
    w_y = 1 / S * best_boxes[..., 2:4]
    converted_bboxes = torch.cat((x, y, w_y), dim=-1)
    predicted_class = predictions[..., :20].argmax(-1).unsqueeze(-1)
    best_confidence = torch.max(predictions[..., 20], predictions[..., 25]).unsqueeze(
        -1
    )
    converted_preds = torch.cat(
        (predicted_class, best_confidence, converted_bboxes), dim=-1
    )

    return converted_preds


def cellboxes_to_boxes(out, S=7):
    converted_pred = convert_cellboxes(out).reshape(out.shape[0], S * S, -1)
    converted_pred[..., 0] = converted_pred[..., 0].long()
    all_bboxes = []

    for ex_idx in range(out.shape[0]):
        bboxes = []

        for bbox_idx in range(S * S):
            bboxes.append([x.item() for x in converted_pred[ex_idx, bbox_idx, :]])
        all_bboxes.append(bboxes)

    return all_bboxes

def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)


def load_checkpoint(checkpoint, model, optimizer):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

# YOLO

In [11]:
import torch
import torch.nn as nn

In [12]:
from torchvision.models import resnet50

# Load pretrained ResNet-50

class CNNBlock(nn.Module):
    def __init__(self,in_channels,out_channels,**kwargs):
        
        super(CNNBlock,self).__init__()
        
        self.conv=nn.Conv2d(in_channels,out_channels,bias=False,**kwargs) #bias= False because of batch norm
        self.batchnorm=nn.BatchNorm2d(out_channels)
        self.leakyrelu=nn.LeakyReLU(0.1)

    def forward(self,x):
        return self.leakyrelu(self.batchnorm(self.conv(x)))
        

class Yolov1(nn.Module):
    def __init__(self, in_channels=3,**kwargs):
        super(Yolov1,self).__init__()
        resnet = resnet50(weights='ResNet50_Weights.DEFAULT')
        self.backbone = nn.Sequential(*list(resnet.children())[:-2]) 
        
        self.in_channels=in_channels
        
        self.fcs= self._create_fcs(**kwargs)

    def forward(self,x):
        x=self.backbone(x)
        # x = torch.flatten(x, start_dim=1) 
        return self.fcs(torch.flatten(x,start_dim=1))
    
    def _create_fcs(self,split_size,num_boxes,num_classes):
        S,B,C=split_size,num_boxes,num_classes
        return nn.Sequential(
            
            nn.Flatten(),
            nn.Linear(2048 * S * S, 496),
            nn.Dropout(0.1),
            nn.LeakyReLU(0.1),
            #YOLO STYLE
            nn.Linear(496,S * S * (C + B * 5)), #(S,S,30) where C+B+S=30
        )



        
                
    
        


In [13]:
# def test(split_size=7,num_boxes=2,num_classes=20):
#     model=Yolov1(split_size=split_size,num_boxes=num_boxes,num_classes=num_classes)
#     x=torch.randn((2,3,448,448))
#     print(model(x).shape)
# test()

#LOSS FUNCTION as per YOLOv1

In [14]:
class YoloLoss(nn.Module):
    

    def __init__(self, S=7, B=2, C=20):
        super(YoloLoss, self).__init__()
        self.mse = nn.MSELoss(reduction="sum")

       
       
        self.S = S
        self.B = B
        self.C = C

        
        self.lambda_noobj = 0.5
        self.lambda_coord = 5

    def forward(self, predictions, target):
        
        predictions = predictions.reshape(-1, self.S, self.S, self.C + self.B * 5)

        
        iou_b1 = intersection_over_union(predictions[..., 21:25], target[..., 21:25])
        iou_b2 = intersection_over_union(predictions[..., 26:30], target[..., 21:25])
        ious = torch.cat([iou_b1.unsqueeze(0), iou_b2.unsqueeze(0)], dim=0)

        
        iou_maxes, bestbox = torch.max(ious, dim=0)
        exists_box = target[..., 20].unsqueeze(3) 
        #BOX CO-ORDINATES
        box_predictions = exists_box * (
            (
                bestbox * predictions[..., 26:30]
                + (1 - bestbox) * predictions[..., 21:25]
            )
        )

        box_targets = exists_box * target[..., 21:25]

        # Take sqrt of width, height of boxes to ensure that
        box_predictions[..., 2:4] = torch.sign(box_predictions[..., 2:4]) * torch.sqrt(
            torch.abs(box_predictions[..., 2:4] + 1e-6)
        )
        box_targets[..., 2:4] = torch.sqrt(box_targets[..., 2:4])

        box_loss = self.mse(
            torch.flatten(box_predictions, end_dim=-2),
            torch.flatten(box_targets, end_dim=-2),
        )

        # FOR OBJECT LOSS

        # pred_box is the confidence score for the bbox with highest IoU
        pred_box = (
            bestbox * predictions[..., 25:26] + (1 - bestbox) * predictions[..., 20:21]
        )

        object_loss = self.mse(
            torch.flatten(exists_box * pred_box),
            torch.flatten(exists_box * target[..., 20:21]),
        )

        # FOR NO OBJECT LOSS
        #max_no_obj = torch.max(predictions[..., 20:21], predictions[..., 25:26])
        #no_object_loss = self.mse(
        #    torch.flatten((1 - exists_box) * max_no_obj, start_dim=1),
        #    torch.flatten((1 - exists_box) * target[..., 20:21], start_dim=1),
        #)

        no_object_loss = self.mse(
            torch.flatten((1 - exists_box) * predictions[..., 20:21], start_dim=1),
            torch.flatten((1 - exists_box) * target[..., 20:21], start_dim=1),
        )

        no_object_loss += self.mse(
            torch.flatten((1 - exists_box) * predictions[..., 25:26], start_dim=1),
            torch.flatten((1 - exists_box) * target[..., 20:21], start_dim=1)
        )

        # CLASS LOSS

        class_loss = self.mse(
            torch.flatten(exists_box * predictions[..., :20], end_dim=-2,),
            torch.flatten(exists_box * target[..., :20], end_dim=-2,),
        )

        loss = (
            self.lambda_coord * box_loss  # first two rows in paper
            + object_loss  # third row in paper
            + self.lambda_noobj * no_object_loss  # forth row
            + class_loss  # fifth row
        )

        return loss

# DATASET

In [15]:
import pandas as pd
from PIL import Image

class VOCDataset(torch.utils.data.Dataset):
    def __init__(
        self,csv_file,img_dir,label_dir,S=7,B=2,C=20,transform=None
    ):
        self.annotations=pd.read_csv(csv_file)
        self.img_dir=img_dir
        self.label_dir=label_dir
        self.transform=transform
        self.S=S
        self.B=B
        self.C=C

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self,index):
        label_path=os.path.join(self.label_dir,self.annotations.iloc[index,1]) #col1 is text file
        boxes=[]
        with open(label_path) as f:
            for label in f.readlines():
                class_label,x,y,width,height=[
                    float(x) if float(x) != int(float(x)) else int(x)
                    for x in label.replace("\n", "").split()
                ]
                boxes.append([class_label,x,y,width,height])
        img_path=os.path.join(self.img_dir,self.annotations.iloc[index,0])
        image=Image.open(img_path)
        boxes=torch.tensor(boxes)

        if self.transform:
            # print(f'yes')
            image, boxes=self.transform(image,boxes)
        label_matrix= torch.zeros((self.S,self.S,self.C + 5*self.B))
        for box in boxes:
            class_label,x,y,width,height=box.tolist()
            class_label=int(class_label)
            i,j=int(self.S*y), int(self.S * x)
            x_cell,y_cell= self.S * x - j, self.S * y - i      
            width_cell,height_cell=(
                width * self.S,
                height * self.S
            )
            if label_matrix[i,j,20]==0:
                label_matrix[i,j,20]=1
                box_coordinates= torch.tensor(
                    [x_cell,y_cell,width_cell,height_cell]
                )
                label_matrix[i,j,21:25]= box_coordinates
                label_matrix[i,j,class_label]=1 #just like one hot setting the right class label as high
        return image,label_matrix
            

# TRAIN

In [16]:
import torchvision.transforms as transforms
import torch.optim as optim
import torchvision.transforms.functional as FT

from tqdm import tqdm
from torch.utils.data import DataLoader


seed=123
torch.manual_seed(seed)

In [17]:
import torch
LEARNING_RATE=1e-5
DEVICE="cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE=16
WEIGHT_DECAY=2e-4

EPOCHS=2000
NUM_WORKERS=2
PIN_MEMORY=True
LOAD_MODEL=False
LOAD_MODEL_FILE="/kaggle/working/overfit.pth.tar"
# LOAD_MODEL_FILE="/kaggle/input/resnet_yolo/pytorch/default/1/overfit.pth.tar"
IMG_DIR="/kaggle/input/pascalvoc-yolo/images"
LABEL_DIR="/kaggle/input/pascalvoc-yolo/labels"


In [18]:
class Compose(object):
    def __init__(self,transforms):
        self.transforms=transforms

    def __call__(self,img,bboxes):
        for t in self.transforms:
            img,bboxes=t(img),bboxes
        return img,bboxes

transform= Compose([transforms.Resize((224,224)),transforms.ToTensor()])

In [19]:
def train_fn(train_loader,model,optimizer,loss_fn):
    loop=tqdm(train_loader,leave=True)
    mean_loss=[]

    for batch_idx,(x,y) in enumerate(loop):
        # print('hi')
        x,y=x.to(DEVICE),y.to(DEVICE)
        out=model(x)
        # print(f' our shape {out.shape}')
        # print(f'y shape is {y.shape}')
        loss= loss_fn(out,y)
        # print(loss)
        
        optimizer.zero_grad()
        torch.autograd.set_detect_anomaly(True)
        loss.backward()
        optimizer.step()
        # scheduler.step()
        mean_loss.append(loss.item())

        loop.set_postfix(loss= loss.item())
        
    print(f"Mean loss was {sum(mean_loss)/len(mean_loss)}")
    

In [20]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
model = Yolov1(split_size=7, num_boxes=2, num_classes=20).to(DEVICE)

train_dataset = VOCDataset(
    "/kaggle/input/pascalvoc-yolo/100examples.csv",
    transform=transform,
    img_dir=IMG_DIR,
    label_dir=LABEL_DIR,
)

test_dataset = VOCDataset(
    "/kaggle/input/pascalvoc-yolo/test.csv", transform=transform, img_dir=IMG_DIR, label_dir=LABEL_DIR,
)

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=PIN_MEMORY,
    shuffle=True,
    drop_last=True,
)


optimizer = optim.Adam(
    model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY
)
# scheduler = torch.optim.lr_scheduler.OneCycleLR(
#     optimizer, max_lr=3e-3,
#     steps_per_epoch=len(train_loader),
#     epochs=EPOCHS
# )
loss_fn = YoloLoss()

if LOAD_MODEL:
    load_checkpoint(torch.load(LOAD_MODEL_FILE), model, optimizer)


# print(f'hhhhh {len(train_loader)}')

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=PIN_MEMORY,
    shuffle=True,
    drop_last=True,
)
mean_max=0
for epoch in range(EPOCHS):
    for x, y in train_loader:
       x = x.to(DEVICE)
       
        
       # for idx in range(8):
       #     bboxes = cellboxes_to_boxes(model(x))
       #     print(f'length {len(bboxes)}')
       #     bboxes = non_max_suppression(bboxes[idx], iou_threshold=0.5, threshold=0.4, box_format="midpoint")
       #     plot_image(x[idx].permute(1,2,0).to("cpu"), bboxes)

       # import sys
       # sys.exit()

    pred_boxes, target_boxes = get_bboxes(
        train_loader, model, iou_threshold=0.5, threshold=0.4
    )
    
    mean_avg_prec = mean_average_precision(
        pred_boxes, target_boxes, iou_threshold=0.5, box_format="midpoint"
    )
    print(f"Train mAP: {mean_avg_prec}")
    if mean_avg_prec>mean_max:
       mean_max=mean_avg_prec
    
    
       checkpoint = {
           "state_dict": model.state_dict(),
           "optimizer": optimizer.state_dict(),
       }
       torch.save(checkpoint,LOAD_MODEL_FILE)
       print(f'saved max mean average precision{mean_max}')
       # save_checkpoint(checkpoint, filename=LOAD_MODEL_FILE)
       import time
       time.sleep(10)
       

    train_fn(train_loader, model, optimizer, loss_fn)
print(f'highest map is {mean_max}')

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 222MB/s]


Train mAP: 0.0


100%|██████████| 6/6 [00:02<00:00,  2.94it/s, loss=807]

Mean loss was 1245.8047587076824


Train mAP: 0.0


...

100%|██████████| 6/6 [00:01<00:00,  3.80it/s, loss=4.8]

Mean loss was 6.940947930018107


Train mAP: 0.49201640486717224


100%|██████████| 6/6 [00:01<00:00,  3.81it/s, loss=6.46]

Mean loss was 6.982507228851318
highest map is 0.58732008934021


In [21]:
# torch.cuda.empty_cache()

In [22]:
# import torch
# from torch.utils.data import DataLoader
# import matplotlib.pyplot as plt

# # ======== Load Your Model and Utilities ========= #
# model = Yolov1(split_size=7, num_boxes=2, num_classes=20).to(DEVICE)
# optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# # Load checkpoint
# checkpoint = torch.load(LOAD_MODEL_FILE, map_location=DEVICE)
# model.load_state_dict(checkpoint['state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer'])

# model.eval()  # set to evaluation mode

# # ======== Prepare Test Dataset and DataLoader ========= #
# test_dataset = VOCDataset(
#     "/kaggle/input/pascalvoc-yolo/test.csv",
#     transform=transform,
#     img_dir=IMG_DIR,
#     label_dir=LABEL_DIR,
# )

# test_loader = DataLoader(
#     dataset=test_dataset,
#     batch_size=1,  # one image at a time for visualization
#     shuffle=True,
# )

# # ======== Inference and Visualization ========= #
# num_samples = 5  # Number of test images to visualize

# with torch.no_grad():
#     for idx, (x, y) in enumerate(test_loader):
#         x = x.to(DEVICE)

#         predictions = model(x)
#         boxes = cellboxes_to_boxes(predictions)

#         bboxes = non_max_suppression(
#             boxes[0], iou_threshold=0.5, threshold=0.4, box_format="midpoint"
#         )

#         plot_image(x[0].permute(1, 2, 0).cpu(), bboxes)

#         if idx + 1 == num_samples:
#             break

  




In [23]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches